## 판례문 chunking

In [ ]:
from konlpy.tag import Okt
import pandas as pd
import re
import requests

def comma_parsing(okt : Okt, text: str) -> list[str]: 
    chunks = re.split(r"\,[^0-9]", text)
    chunks_info = []

    # 구/절 구분
    for i in range(len(chunks)):
        is_clause = False
        for pos in okt.pos(chunks[i]):
            # 한국어 문장 기본 구조는 : 주어 + 목적어 + 서술어, 주어 + 보어 + 서술어, 주어 + 서술어
            # 서술어 : 동사, 형용사, 서술격 조사 -이다
            # 동사, 형용사 존재 시 완전한 문장인 절로 간주
            # 예외적인 경우는 무시. 예외에 해당하는 문장은 단순하고 짧은 문장일 것이므로 결과에 지장이 없을 것으로 예상.
            if (pos[1]) in ["Verb", "Adjective"]: 
                is_clause = True
                break
        chunks_info.append(is_clause)

    # 병합
    # 콤마는 제거된다
    data = []
    clause = ''
    for i in range(len(chunks_info)):
        is_clause = chunks_info[i]
        # 절(완전한 문장)
        if is_clause:
            clause += " " + chunks[i]
            data.append(clause.strip())
            clause = ''
        # 명사구
        else:
            clause += " " + chunks[i]

    # test log
    # for i in range(len(chunks)):
    #   print(f"{chunks[i]} : {chunks_info[i]}")

    return data 
                

def preprocessing_content(url) -> pd.DataFrame:
    try:
        content = requests.get(url).json()["PrecService"]["판례내용"]
    except requests.exceptions.RequestException as e:
        print(e)
        return

    # parsed = re.split(r"【이\s+유】", content)

    data = []

    # lines = parsed[1].split("<br/>")
    lines = content.split("<br/>")
    for line in lines:
        line = line.strip()

        line = re.sub(r"^([0-9]{1,2}|[가-하])[).]", "", line)  # 목차 제거
        line = re.sub(r"\.", "", line)  # 온점 제거
        line = re.sub(r"[‘’“”]", "", line)  # 따옴표 제거
        line = re.sub(r"·", "", line) # · 기호 제거
        line = re.sub(f"[\u2460-\u2473]", "", line) # [\u2460-\u2473] : 1-20 ①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳ [\u3251-\u32BF] : 21-50
        line = re.sub(f"[\u326e-\u327b]", "", line) # [\u326e-\u327b] ㉮-㉻

        data += comma_parsing(Okt(), line)

    df = pd.DataFrame(data, columns=["text"])
    return df

## 참조 법령 스캔

## Query - Pos 쌍 생성
query : 법령  
pos : (연관) 판례문 내용 문장